<div style="font-family: 'Gen Jyuu Gothic Monospace Medium', 'Noto Sans TC'; font-size: 130%; text-align: center; color: Plum;">
<br>

# <font color='Plum'><b>詳談Structural Pattern Matching</b></font>
</div>
<!-- <br> -->
<div style="font-family: Inconsolata, 'Noto Sans TC'; font-size: 135%;">

* Python 3.10最重大的新增功能。
* 類似C-like系列語言的switch/case，但<font color='Gold'><b>更強大</b></font>(truly awesome)。
* 建議找時間熟悉使用。

<div style="text-align:center"><img src="https://i.imgur.com/MqsfB7X.jpg" width="650"/></div>

In [3]:
import sys

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'
'InteractiveShell set.'
sys.version

'InteractiveShell set.'

'3.11.1 (main, Jan 28 2023, 18:47:50) [GCC 11.3.0]'

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

## <font color='SteelBlue'><b>多重條件時Python的傳統寫法</b></font>

* 3.9版以前，Python沒有switch/case，多重條件只能if/elif。筆者覺得這結構「賣相(非功能)不佳」，是Python語法上少數暗點之一。
* 當然，這是否為暗點見仁見智，總之個人不大喜歡就是。筆者對此語法有意見，主要原因是<font color='RosyBrown'>'=='</font>右邊的「<font color='RosyBrown'>literal(定數)</font>」或變數無法對齊。
* 以下用一個最簡單的例子複習Python傳統的if/elif語法。

In [86]:
http_code = '418'

if http_code == '200':  # 這個'200'和下面的'401', '418'等地位相同卻對不齊。
    print('OK')
elif http_code == '401' or http_code == '403' or http_code == '404':
    print('Not Found')
elif http_code == '418':
    print("I'm a teapot")
elif http_code == "500":
    print('Internal Server Error')
else:
    print("Undefined code")

I'm a teapot


<div style="text-align:center"><img src="https://i.imgur.com/YDpGhCz.jpg" width="650"/></div>

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

## <font color='SteelBlue'><b>多重條件的新語法</b></font>

* 終於終於，Python從3.10版起可以用類似switch/case的結構了。這個新語法稱為<font color='Gold'><b>Structural Pattern Matching</b></font>，本文簡稱Pattern Matching或Matching。
* 以下用Pattern Matching改寫樓上的if/elif。結構簡潔，定數漂亮對齊。

In [7]:
http_code = '418'

match http_code:
    # match和case之間不可以有任何指令。可試打開下面註解掉的任何一行。
    # print('Pattern Matching test.')
    # if True:
    #     pass
    case '200':                  # '200'和下面的'401', '418'等對齊，整齊多了。
        print ('OK')
    case '401' | '403' | '404':  # 可以用'|'表示「或」。
        print('Not allowed')
    case '418':
        print("I'm a teapot")
    # 2個cases之間也不能有指令。
    # name = "Alex"
    case '500':
        print('Internal Server Error')
    case _:                  # 相當於if/elif中的else。
        print("Undefined code")

I'm a teapot


<div style="text-align:center"><img src="https://i.imgur.com/gjWU5sF.jpg" width="650"/></div>

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

## <font color='SteelBlue'><b>Cool，但談不上啥特異功能吔</b></font>

* 有人也許會說，這個Pattern Matching很漂亮，可只是語法上的改進或語法甜頭(syntactic sugar)而已，並非「新增功能」。
* 沒錯，Pattern Matching能耐如果僅及於此，的確不算新增功能。但上面這段code只是它的基本款，牛刀小試耳。
* Pattern Matching絕對不只是C中的switch/case。它功能比switch強大得多，不過學習門檻較高，關卡堡壘不少。必須耐心逐個攻破，方得全貌。

<div style="text-align:center"><img src="https://i.imgur.com/WTQTz5U.jpg" width="650"/></div>

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

## <font color='SteelBlue'><b>Bridge Not so Far</b></font>

* 以下筆者將步步為營，謹慎求證，不會重蹈[「市場花園行動」](https://zh.wikipedia.org/wiki/%E5%B8%82%E5%A0%B4%E8%8A%B1%E5%9C%92%E8%A1%8C%E5%8B%95)覆轍。<font color='RosyBrown'>橋雖遠，終有到達一天</font>。
* 搶占Pattern Matching灘頭堡的[「諾曼第登陸」](https://zh.wikipedia.org/wiki/%E8%AF%BA%E6%9B%BC%E5%BA%95%E6%88%98%E5%BD%B9)開始：

<div style="text-align:center"><img src="https://i.imgur.com/5PtBIFZ.jpg" width="650"/></div>

<div style="text-align:center"><img src="" width="650"/></div>


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

### <font color='DarkSalmon'><b>1. 寶劍海灘：match和case非保留字</b></font>

* 3.10版雖然新增了match/case的matching結構，但match和case這兩個字[只是soft keywords，不是hard keywords](https://www.python.org/dev/peps/pep-0622/)，意思是這兩個key words<font color='Gold'>並未列為保留字</font>(reserved words)。這個聰明措施使得3.10版和之前的版本相容，各位可以放心使用match和case作為變數。
* Pattern Matching結構本身的<font color='RosyBrown'>match</font>和<font color='RosyBrown'>case</font>也不是變數。

In [5]:
match = 'I am a match.'
print(f'Before: {match = }')
case = 'I am a case.'
print(f'Before: {case = }')
match match:   # 前面的match是Pattern Matching的關鍵字，後面的match則是變數。
# matches case:
    case 'I am a match.':
        print('matches case 0')
    case 'I am a case.':
        print('matches case 1')
    case _:
        print('no match')


if 'match' in globals():
    print(f'After : {match = }')
if 'case' in locals():   # 用globals()和locals()判斷都行。
    print(f'After : {case = }')
# 經過Pattern Matching之後，match和case的值不變。

Before: match = 'I am a match.'
Before: case = 'I am a case.'
matches case 0
After : match = 'I am a match.'
After : case = 'I am a case.'


<div style="text-align:center"><img src="https://i.imgur.com/i1ArczU.jpg" width="650"/></div>

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

### <font color='DarkSalmon'><b>2. 朱諾海灘：pattern內可以有變數</b></font>
* Pattern Matching就是可以用'<font color='Chartreuse'>pattern</font>'來比對。
* 意思是：case的後面除了literal「定數」外，還可以是個pattern。
* pattern比對順序由上至下，而且最多只有1個匹配。前面的pattern匹配到，就自動跳離match結構，不再往後比對。因此case內不必也不能有break。
* 本節重點是：<font color='Gold'><b>pattern內可以使用變數</b></font>。這是Pattern Matching比switch/case強大的一大原因。
* 下例的pattern是比對一個dict。

In [8]:
dict_to_match = {'name': 'Becca', 'age': 28}
# dict_to_match = {'name': 'Becca', 'age': 6}
match dict_to_match:      # 所謂Pattern Matching，就是可以用'pattern'來比對。
    # case後面的就是pattern。
    case {'name': 'Alex', 'age': 38}:   # ．此pattern比對'Alex'和38。
        print('matches case 0')         #   匹配到會自動跳離，不須也不能寫break。
    case {'name': name, 'age': age}:    # ．以變數name和age比對任何名稱和年齡。
        print('matches case 1')         #   value的變數名稱不一定要和key名稱相同。
    case {'name': name1, 'age': age1}:  # ．本pattern和case 1的pattern完全相同。因
        print('matches case 2')         #   case 1較優先，所以本pattern永遠不會有匹配。
    case _:
        print('no match')

matches case 1


<div style="text-align:center"><img src="https://i.imgur.com/6gBZSif.jpg" width="650"/></div>

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

### <font color='DarkSalmon'><b>3. 黃金海灘：pattern不必包含dict的全部keys</b></font>
* 要比對的物件(即match後面的變數)如果是dict，<font color='Gold'>pattern不一定要包含該dict所有的keys</font>。
* 可少卻不可多。如果pattern多出一個原dict沒有的key，就無法匹配。
* 原dict和pattern的keys順序可以不一樣。

In [51]:
def match(obj: object) -> tuple:
    match obj:
        case {'id': id,      # 本pattern匹配任何有這些keys的dict。
              'info': {'name': name, 'age': age}}:  # pattern不須包括dict的全部keys。
            match_obj = {'id': id, 'info': {'name': name, 'age': age}}
            prompt = r"pattern          : {'id': id, 'info': {'name': name, 'age': age}}"
            case = f'matches case 0   : {match_obj}' 
            return_value = prompt, case
        case {'id': id,       # 比對dict時，pattern的key可少卻不可多。
              'name': name,   # 每次傳入的person，都沒有一個'edu'的key，
              'edu': edu,     # 所以本pattern無法匹配任何person。
              'bmi': bmi}:
            match_obj = {'id': id, 'name': name, 'edu': edu, 'bmi': bmi}
            prompt = r"pattern         : {'id': id, 'name': name, 'edu': edu, 'bmi': bmi}"
            case = f'matches case 1  : {match_obj}'
            return_value = prompt, case
        case {'name': {'first': first, 'last': last,}, 'id': id}:   # keys的順序無所謂。
            match_obj = {'name': {'first': first, 'last': last}, 'id': id}
            prompt = r"pattern         : {'name': {'first': first, 'last': last}, 'id': id}"
            case = f'matches case 2  : {match_obj}'
            return_value = prompt, case
        case _:  # 以上皆非
            match_obj = 'no match'
            prompt = r'pattern         : _'
            case = f'{match_obj}'
            return_value = prompt, case
    return return_value

# 這個persons內的元素是要被比對的物件。
persons = [
    {'id': 1, 'info': {'name': 'Ed', 'age': 49, 'eye': 'black'}},
    {'id': 2, 'name': 'Lotus', 'gender': 'F', 'bmi': 21.2},
    {'id': 3, 'name': {'last': '林', 'first': '蓮'}, 'bmi': 19.3},
]

for person in persons:
    print(f'obj to be matched: {person}')
    print(f'{match(person)[0]}\n{match(person)[1]}\n')

obj to be matched: {'id': 1, 'info': {'name': 'Ed', 'age': 49, 'eye': 'black'}}
pattern          : {'id': id, 'info': {'name': name, 'age': age}}
matches case 0   : {'id': 1, 'info': {'name': 'Ed', 'age': 49}}

obj to be matched: {'id': 2, 'name': 'Lotus', 'gender': 'F', 'bmi': 21.2}
pattern         : _
no match

obj to be matched: {'id': 3, 'name': {'last': '林', 'first': '蓮'}, 'bmi': 19.3}
pattern         : {'name': {'first': first, 'last': last}, 'id': id}
matches case 2  : {'name': {'first': '蓮', 'last': '林'}, 'id': 3}



<div style="text-align:center"><img src="https://i.imgur.com/tHGNOYq.jpg" width="650"/></div>

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

### <font color='DarkSalmon'><b>4. 奧馬哈海灘：list和tuple意義相同</b></font>
* 比對時list和tuple意義完全相同，都是<font color='Gold'>sequence</font>(序列)。
* 所謂sequence，可粗略理解成「<font color='RosyBrown'>可使用索引以及切片</font>」的型別，如list, tuple和str。
* 而Matching機制為了避免產生某些錯誤，在match sequence時排除了str。所以<font color='Gold'>str不歸sequence管，是獨立比對的</font>。

In [52]:
def match(obj: object) -> tuple:
    match obj:
        case (item1, item2):
            match_obj = (item1, item2)
            prompt = r'pattern          : (item1, item2)'
            case = f'matches case 0   : {match_obj}'
            return_value = prompt, case            
        case [item1, item2, item3]:
            match_obj = [item1, item2, item3]
            prompt = r'pattern          : [item1, item2, item3]'
            case = f'matches case 1   : {match_obj}'
            return_value = prompt, case            
        case 'pi':
            match_obj = 'pi'
            prompt = r"pattern          : 'pi'"
            case = f'matches case 2   : {match_obj}'
            return_value = prompt, case            
        case _:
            match_obj = 'no match'
            prompt = r'pattern          : _'
            case = f'{match_obj}'
            return_value = prompt, case                        
    return return_value


objs = (
    [0, 1],                                       # 2-item list
    (True, False, None),                          # 3-item tuple
    [[0, 1, 1, 2, 3, 5], (2, 3, 5, 7, 11, 13)],   # 2-item list
    (2.71828,),                                   # 1-item tuple(Euler's number, e)
    ['春分', '夏至', '秋分', '冬至'],           # 4-item list
    (),                                           # 0-item tuple
    "['Isaac Newton', 'Albert Einstein']",         # str
)
    
for obj in objs:
    print(f'obj to be matched: {obj}')
    print(f'{match(obj)[0]}\n{match(obj)[1]}\n')

obj to be matched: [0, 1]
pattern          : (item1, item2)
matches case 0   : (0, 1)

obj to be matched: (True, False, None)
pattern          : [item1, item2, item3]
matches case 1   : [True, False, None]

obj to be matched: [[0, 1, 1, 2, 3, 5], (2, 3, 5, 7, 11, 13)]
pattern          : (item1, item2)
matches case 0   : ([0, 1, 1, 2, 3, 5], (2, 3, 5, 7, 11, 13))

obj to be matched: (2.71828,)
pattern          : _
no match

obj to be matched: ['春分', '夏至', '秋分', '冬至']
pattern          : _
no match

obj to be matched: ()
pattern          : _
no match

obj to be matched: ['Isaac Newton', 'Albert Einstein']
pattern          : _
no match



<div style="text-align:center"><img src="https://i.imgur.com/bR5Ztp7.png" width="650"/></div>

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

### <font color='DarkSalmon'><b>5. 猶他海灘：支援萬用字元</b></font>
* Pattern Matching支援wildcards(萬用或百搭)字元比對，如<font color='Gold'><b>[x, y, *rest]</b></font>或<font color='Gold'><b>(x, y, *rest)</b></font>。意思是：這2個patterns要比對2-N個元素的sequence(list or tuple)。rest可以改用其他合法變數名稱如<font color='RosyBrown'>[x, y, *others]</font>或者<font color='RosyBrown'>(x, y, *others)</font>。
* <font color='Gold'><b>*rest</b></font>其實就是Python著名的[unpacking秘技](https://www.learncodewithmike.com/2019/12/python-unpacking.html)。
* <font color='Gold'><b>[x, y, *&#95;]</b></font>或<font color='Gold'><b>(x, y, *&#95;)</b></font>則較為特殊。以筆者測試後的粗淺理解，這個寫法意思是比對一個最少有2個元素的sequence，可是第3個以後的元素卻不會unpack，或者說它也有unpack，但方式和一般不同。
* 至於dict的比對，也支援wildcard<font color='Gold' ><b>&#42;&#42;rest</b></font>，但不允許用<font color='Gold' ><b>**&#95;</b></font>。

In [53]:
def match(obj: object) -> tuple:
    match obj:
        case (item1, item2):
            match_obj = (item1, item2)
            prompt = r'pattern         : (item1, item2)'
            case = f'matches case 0  : {match_obj}'
            return_value = prompt, case                        
        case [item1, item2, *others]:  # 請試註解掉這個case區塊，看結果如何。
            match_obj = [item1, item2, *others]
            prompt = r'pattern         : [item1, item2, *others]'
            case = f'matches case 1  : {match_obj}'
            return_value = prompt, case                        
        case (item1, item2, *_):
            match_obj = (item1, item2, *_)     # 這裡'_'就好，不要'*_'再unpack一次。
            prompt = r'pattern         : (item1, item2, *_)'
            case = f'matches case 2  : {match_obj}'
            return_value = prompt, case                                    
        case {"name": name, **other_pairs}:   # 用**other_pairs來百搭dict其他的pairs。
            match_obj = (name, other_pairs)
            prompt = r'pattern         : {"name": name, **other_pairs}'
            case = f'matches case 3  : {match_obj}'
            return_value = prompt, case
        # case {"name": name, **_}:             # invalid syntax
            # match_obj = ("name": name, _)
            # prompt = r'pattern         : {"name": name, **_}'
            # case = f'matches case 4  : {match_obj}'
            # return_value = prompt, case                     
        case _:
            match_obj = 'no match'
            prompt = r'pattern         : _'
            case = f'{match_obj}'
            return_value = prompt, case                     
    return return_value

objs = (
    ('Σ',),                      # 1-item tuple(no match)
    ['Σ', 'Φ'],                  # 2-item list(matches case 0)
    ('Σ', 'Φ', 'Θ', 'Ψ', 'Ω'),   # 5-item tuple(matches case 1)
    {'name': '王羲之', 'category': '書法', 'piece': '蘭亭集序'},  # dict(matches case 3)
)

for obj in objs:
    print(f'obj to be matched: {obj}')
    print(f'{match(obj)[0]}\n{match(obj)[1]}\n')

obj to be matched: ('Σ',)
pattern         : _
no match

obj to be matched: ['Σ', 'Φ']
pattern         : (item1, item2)
matches case 0  : ('Σ', 'Φ')

obj to be matched: ('Σ', 'Φ', 'Θ', 'Ψ', 'Ω')
pattern         : [item1, item2, *others]
matches case 1  : ['Σ', 'Φ', 'Θ', 'Ψ', 'Ω']

obj to be matched: {'name': '王羲之', 'category': '書法', 'piece': '蘭亭集序'}
pattern         : {"name": name, **other_pairs}
matches case 3  : ('王羲之', {'category': '書法', 'piece': '蘭亭集序'})



<div style="text-align:center"><img src="https://i.imgur.com/0G0ZEFZ.jpg" width="650"/></div>

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

### <font color='DarkSalmon'><b>6. 瑟堡：設定變數型別</b></font>
* pattern中的變數如果沒有指定型別，會匹配任何型別物件。
* 可用<font color='Gold'><b>str(var)</b></font>, <font color='Gold'><b>int(var)</b></font>, <font color='Gold'><b>float(var)</b></font>等指明型別。

In [54]:
def match(obj: object) -> tuple:
    match obj:
        case [str(item1), str(item2)]:  # 比對1個sequence，內有2個型別為str的元素。
            match_obj = [item1, item2]
            prompt = r'pattern         : [str(item1), str(item2)]'
            case = f'matches case 0  : {match_obj}'
            return_value = prompt, case                        
            
        case (int(num1), float(num2)):  # 比對1個sequence，2個元素，分別是int和float。
            match_obj = (num1, num2)
            prompt = r'pattern         : (int(num1), float(num2))'
            case = f'matches case 1  : {match_obj}'
            return_value = prompt, case                        
            
        case _:                         # 以上皆非
            match_obj = 'no match'
            prompt = r'pattern         : _'
            case = f'{match_obj}'
            return_value = prompt, case                        
        
    return return_value

objs = [
    ('沈括', '張衡'), # 華夏古代大科學家(匹配case 0)
    [1644, 1912],      # 1644年明亡，1912年清亡(無匹配)
    [1.3247, 1.618],   # 塑膠數、黃金比例(無匹配)
    (0, 3.1415927),    # 南北朝時代祖沖之計算的Pi精確到小數點後7位，領先全球一千多年(匹配case 1)
    (1.41421, 6174)    # 畢達哥拉斯常數, 黑洞數(無匹配)
]

for obj in objs:
    print(f'obj to be matched: {obj}')
    print(f'{match(obj)[0]}\n{match(obj)[1]}\n')

obj to be matched: ('沈括', '張衡')
pattern         : [str(item1), str(item2)]
matches case 0  : ['沈括', '張衡']

obj to be matched: [1644, 1912]
pattern         : _
no match

obj to be matched: [1.3247, 1.618]
pattern         : _
no match

obj to be matched: (0, 3.1415927)
pattern         : (int(num1), float(num2))
matches case 1  : (0, 3.1415927)

obj to be matched: (1.41421, 6174)
pattern         : _
no match



<div style="text-align:center"><img src="https://i.imgur.com/XpGZbfZ.jpg" width="650"/></div>

In [55]:
def match(obj: object) -> tuple:
    match obj:
        # 請試註解下面的case，看結果如何。
        case set():               # 比對set要用set()，不可以{stuff, ...}。
            match_obj = set()     # 此pattern雖匹配任意的set，但抓不到set內元素。
            prompt = r'pattern         : set()'
            case = f'matches case 0  : {match_obj}'
            return_value = prompt, case
        # 上下兩個patterns都匹配任意元素的set，所以下面的pattern永遠沒有機會大展身手。
        case set(stuff):        # 設定變數，就可以抓到set的所有元素。
            match_obj = stuff   # 這裡的set()只接受一個參數，這參數又稱'sub-pattern'。此參數不能有type hint。
            prompt = r'pattern         : set(stuff)'
            case = f'matches case 1  : {match_obj}'
            return_value = prompt, case                        
        case _:
            match_obj = 'no match'
            prompt = r'pattern         : _'
            case = f'{match_obj}'
            return_value = prompt, case                        
    return return_value

objs = (
    set(),       
    {'Spagetti'},                                    # 意粉
    {'Spagetti', 'Macaroni'},                        # 意粉, 通粉
    {'Spagetti', 'Macaroni', 'Fusilli'},             # 意粉, 通粉, 螺旋粉
    {'Spagetti', 'Macaroni', 'Fusilli', 'Lasagna'},  # 意粉, 通粉, 螺旋粉, 千層麵
)

for obj in objs:
    print(f'obj to be matched: {obj}')
    print(f'{match(obj)[0]}\n{match(obj)[1]}\n')

obj to be matched: set()
pattern         : set()
matches case 0  : set()

obj to be matched: {'Spagetti'}
pattern         : set()
matches case 0  : set()

obj to be matched: {'Spagetti', 'Macaroni'}
pattern         : set()
matches case 0  : set()

obj to be matched: {'Spagetti', 'Macaroni', 'Fusilli'}
pattern         : set()
matches case 0  : set()

obj to be matched: {'Spagetti', 'Macaroni', 'Lasagna', 'Fusilli'}
pattern         : set()
matches case 0  : set()



<div style="text-align:center"><img src="https://i.imgur.com/cIEJaNm.jpg" width="650"/></div>

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

### <font color='DarkSalmon'><b>8. 眼鏡蛇行動：class也可比對</b></font>
* 以下用一個自訂類別<font color='Gold'><b>Couple</b></font>示範class的比對。

In [63]:
class Couple():
    def __init__(self, husband: str, wife: str):  # constructor(建構子)
        self.husband: str = husband
        self.wife: str = wife

    def __repr__(self):
        return f"Husband: {self.husband}\tWife: {self.wife}."

    def do_housework(self):   # method(方法)
        print('We are doing housework...')
        return 'doing housework'


def match(obj: object) -> tuple:
    match obj:
        case Couple(husband='Alex', wife='Lotus'):
            match_obj = ('Alex', 'Lotus')
            prompt = r"pattern         : Couple(husband='Alex', wife='Lotus')"
            case = f'matches case 0  : {match_obj}'
            return_value = prompt, case
        case Couple(husband='Arian', wife='Brett'):  #str(wife)):
            match_obj = ('Arian', 'Brett')    # wife)
            prompt = r"pattern         : Couple(husband='Arian', wife=wife)"
            case = f'matches case 1  : {match_obj}'
            return_value = prompt, case            
        case Couple(he=str(husband), she='Diana'):
        # case Couple(she='Diana'):
            match_obj = ('doing housework.......', )
            prompt = r"pattern         : Couple(do_housework='doing housework')"
            case = f'matches case 2  : {match_obj}'
            return_value = prompt, case
        # case Couple(do_housework='doing housework'):  # TODO:
            # match_obj = ('Diana', )
            # print(f'matches case 2  : {match_obj}')
            # print("pattern         : Couple(husband=husband, wife='Lotus')" '\n')
            # print("pattern         : Couple(she='Diana')" '\n')
            # print("pattern         : Couple(do_housework='doing housework')" '\n')
        case Couple(husband=str(husband), wife=str(wife)):
            match_obj = (husband, wife)
            # print(f'matches case 3  : {match_obj}')
            # print('pattern         : Couple(husband=husband, wife=wife)' '\n')
            prompt = r'pattern         : Couple(husband=husband, wife=wife)'
            case = f'matches case 3  : {match_obj}'
            return_value = prompt, case
        case _:
            match_obj = 'no match'
            # print(match_obj)
            # print('pattern         : _' '\n')
            # pass
            prompt = r'pattern         : _'
            case = f'{match_obj}'
            return_value = prompt, case        
        
    return return_value

couple1 = Couple('Alex', 'Lotus')  # case 1
couple2 = Couple('Chris', 'Diana')  # case 2
couple3 = Couple('Ethan', 'Fiona')  # case 3
couple4 = Couple('Grant', 2021)     # no match
objs = (couple1, couple2, couple3, couple4)

for obj in objs:
    print(f'obj to be matched: {obj}')
    print(f'{match(obj)[0]}\n{match(obj)[1]}\n')

obj to be matched: Husband: Alex	Wife: Lotus.
pattern         : Couple(husband='Alex', wife='Lotus')
matches case 0  : ('Alex', 'Lotus')

obj to be matched: Husband: Chris	Wife: Diana.
pattern         : Couple(husband=husband, wife=wife)
matches case 3  : ('Chris', 'Diana')

obj to be matched: Husband: Ethan	Wife: Fiona.
pattern         : Couple(husband=husband, wife=wife)
matches case 3  : ('Ethan', 'Fiona')

obj to be matched: Husband: Grant	Wife: 2021.
pattern         : _
no match



In [61]:
def do_activity(c: Couple):
    match c:
        case Couple("John", "Jane"):
            print("John and Jane are doing an activity together.")
        case Couple("Peter", "Mary"):
            print("Peter and Mary are doing an activity together.")
        case _:
            print("This couple is not recognized.")

c1 = Couple("John", "Jane")
c2 = Couple("Peter", "Mary")
c3 = Couple("Tom", "Susan")

do_activity(c1) # Output: John and Jane are doing an activity together.
do_activity(c2) # Output: Peter and Mary are doing an activity together.
do_activity(c3) # Output: This couple is not recognized.


TypeError: Couple() accepts 0 positional sub-patterns (2 given)

<div style="text-align:center"><img src="https://i.imgur.com/nX2hdnF.gif" width="650"/></div>

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

### <font color='DarkSalmon'><b>9. 法萊斯包圍戰：Guard</b></font>
* pattern的後面可以加if判斷，這個if clause稱為<font color='Gold'><b>guard</b></font>。
* guard的判斷結果為False時，會往下一個case走。
* case後面的變數會先抓值，然後交給guard運用。

In [ ]:
def match(obj: object) -> tuple:
    # match_obj = 'no match'
    match obj:
        case [str(item1), str(item2)] if item1 == item2:  # 2個字串元素必須相同。
            match_obj = [item1, item2]
            print(f'matches case 0  : {match_obj}')
            print(r'pattern         : str(item1), str(item2)] if item1 == item2' '\n')
        case (int(n1), int(n2), int(n3), int(n4), int(n5), int(n6)) \
              if n1 + n2 == n3 and n2 + n3 == n4 and n3 + n4 == n5 and n4 + n5 == n6:
            match_obj = (n1, n2, n3, n4, n5, n6)
            print(f'matches case 1  : {match_obj}')
            print(r'pattern         : (int(n1), int(n2), int(n3), int(n4), int(n5), int(n6)) if n1 + n2 == n3 and n2 + n3 == n4 and n3 + n4 == n5 and n4 + n5 == n6' '\n')
        case (int(n1), int(n2), int(n3), int(n4), int(n5)) if [i**2 for i in range(1, 6)]:
        # case (int(n1), int(n2), int(n3), int(n4), int(n5)) if [1, 4, 9, 16, 25]:
            match_obj = (n1, n2, n3, n4, n5)
            print(f'matches case 2  : {match_obj}')
            print(r'pattern         : (int(n1), int(n2), int(n3), int(n4), int(n5)) if [i**2 for i in range(1, 6)]' '\n')
        case [float(f1), float(f2)] if len(str(f1).split('.')[1]) == 4 and len(str(f2).split('.')[0]) == 6:
            match_obj = (f1, f2)
            print(f'matches case 3  : {match_obj}')
            print(r"pattern         : [float(f1), float(f2)] if len(str(f1).split('.')[1]) == 4 and len(str(f2).split('.')[0]) == 6" '\n')
        case (bool(b1), bool(b2)) if b1 is not b2:
            match_obj = (b1, b2)
            print(f'matches case 4  : {match_obj}')
            print(r"pattern         : (bool(b1), bool(b2()) if b1 is not b2" '\n')
        case {'height': float(h), 'weight': float(w), 'bmi': float(b)} if round(w / h**2, 1) == b:
            match_obj = (h, w, b)
            print(f'matches case 5  : {match_obj}')
            print(r"pattern         : {'height': float(h), 'weight': float(w), 'bmi': float(b)} if round(w / h**2, 1) == b" '\n')
        case _:
            print(match_obj)
            print('pattern         : _' '\n')
            pass
    return match_obj

objs = [
    ('蘇東坡', '蘇東坡'),
    ('蘇東坡', '蘇軾'),      # Q: 蘇東坡就是蘇軾，這是常識，電腦怎麼不懂？  A: 這不是AI喔
    [0, 1, 1, 2, 3, 5],      # Fibonacci sequence
    [1, 4, 9, 16, 25],       # Square Numbers
    (1.5396, 299792.458),    # Lieb's square ice constant, 光速(km/s)
    [True, False],           # toggle
    {'height': 1.68, 'weight': 66.8, 'bmi': 23.7}   # 66.8 / 1.68**2 = 21.7
]

for obj in objs:
    print(f'obj to be matched: {obj}')
    print(f'{match(obj)[0]}\n{match(obj)[1]}\n')

<div style="text-align:center"><img src="https://i.imgur.com/BUuoLMi.jpg" width="650"/></div>

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

### <font color='DarkSalmon'><b>10. 突出部戰役：if/elif大反攻</b></font>
* 剛才把Patatern Matching差點捧上天，難道Matching就此完勝，把if/elif甩開十萬八千里不成？
* 從實務看，Pattern Matching推出時間也不算短了，可if/elif依然廣泛使用，並沒有任何快遭淘汰跡象。
* 再從功能上檢核，究竟有沒有一些地方是Pattern Matching不如傳統if/elif之處，甚至是if/elif能而Matching不能的？
    * 「if/elif能而Matching不能」的情形應該是沒有，最少筆者至今沒有發現。
    * 不過類似如下的if/elif/else(這裡只談可能性，不管實務上合不合理)，以Pattern Matching來做，就顯得為冗長累贅，if/elif更易閱讀。


In [ ]:
# if/elif版
patient = {'gender': 'F', 'age': 39, 'weight': 39, 'height': 178, 
           'antibiotics': ('amoxicillin', 'cefalexin', 'gentamicin', 'doxycycline', 'levofloxacin')}
print('if/elif版：')
if patient['gender'] == 'M':
    print('gender matched.')
elif patient['age'] > 65 or patient['age'] < 20:
    print('age matched.')
elif patient['weight'] > 60 and patient['height'] <= 175:
    print('weight and height matched.')
elif 'gentamicin' in patient['antibiotics']:
    print('antibiotics matched.')
else:
    print('no match.')
 
# ------------------- 
print('\npattern matching版：')
# pattern matching版
patient = {'gender': 'F', 'age': 39, 'weight': 39, 'height': 178, 
           'antibiotics': ('amoxicillin', 'cefalexin', 'gentamicin', 'doxycycline', 'levofloxacin')}

match patient:
    case {"gender": "M"}:
        print("gender matched.")
    case {"age": age} if age > 65 or age < 20:
        print(f"age ({age}) matched.")
    case {"weight": weight, "height": height} if weight > 60 and height <= 175:
        print(f"weight ({weight}) and height ({height}) matched.")
    case {"antibiotics": antibiotics}:
        if "gentamicin" in antibiotics:
            print("antibiotics matched.")
        else:
            print("no match.")


<div style="text-align:center"><img src="https://i.imgur.com/W3y8htB.jpg" width="650"/></div>

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

### <font color='DarkSalmon'><b>11. 戰終：給個小結論</b></font>
* 綜上所述，Pattern Matching這個Python近年來最大的新語法，的確功能強大，值得花點時間精力學習。
* 可是不能將Pattern Matching視為取代<font color='Gold'><b>if/elif</b></font>的工具。if/elif仍絕對有其存在價值。兩者互有優劣，不必尊此而卑彼。

In [1]:
# if/elif版
patient = {'gender': 'M', 'age': 39, 'weight': 69, 'height': 196, 
           'antibiotics': ('amoxicillin', 'cefalexin', 'gentamicin', 'doxycycline', 'levofloxacin')}

print('if/elif版：')
if patient['gender'] == 'M':
    print('male.')
    if patient['age'] >= 65:
        print('senior.')
    elif patient['height'] > 195:
        print('tall.')
    else:
        print('not senior or tall.')
    print('do other stuff.')
else:
    print('female.')

# ------------------- 
print('\npattern matching版：')
# pattern matching版
match patient:
    case {'gender': 'M'}:
        print('male.')
        match patient:
            case {'age': age} if age >= 65:
                print('senior.')
            case {'height': height} if height > 195:
                print('tall.')
            case _:    
                print('not senior or tall.')
        print('do other stuff.')
    case _:
        print('female.')

if/elif版：
male.
tall.
do other stuff.

pattern matching版：
male.
tall.
do other stuff.


<div style="text-align:center"><img src="https://i.imgur.com/M7Sz32o.jpg" width="650"/></div>

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

## <font color='SteelBlue'><b>戰後：順便談一個程式邏輯</b></font>

<div style="text-align:center"><img src="https://i.imgur.com/CzvHJwB.jpg" width="650"/></div>


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

### <font color='DarkSalmon'><b>多重條件的判斷一定要「先特殊後普通」</b></font>
* 連續判斷多個條件而且各條件有「<font color='Gold'>包含</font>」關係時，要把握<font color='Gold'>越specific(特殊)者越先，越general(普通)者越後</font>這個原則。
* 萬一不小心寫反，結果很可能不是我們所預期。
    * <font color='#ff0003'>most specific = 範圍最窄</font>
    * <font color='#f57b20'>most general = 範圍最寬</font>
<div style="text-align:center"><img src="https://i.imgur.com/c4Cy75H.png" width="650"/></div>


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

### <font color='DarkSalmon'><b>Example-1</b></font>
* 假設：
    * 未滿12歲者(範圍最窄)以下的事全不能做。
    * 12歲以上可看輔級電影。
    * 20歲以上可投票。
    * 40歲以上可選總統。
    * 65歲以上(範圍最寬)可退休。

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

### <font color='DarkSalmon'><b>「先普通後特殊」寫法：錯</b></font>

* 再說一次：多重條件時的判斷順序，要從<font color='gold'><b>範圍最窄條件到最寬條件</b></font>，不能倒過來從最寬到最窄。

In [95]:
# 從最general到最specific的寫法錯誤。
ages = (10, 12, 20, 39, 40, 67)
for age in ages:
    print(f'{age}歲：', end='')
    if age >= 12:   # 最general(範圍最寬)
        print('可看輔級電影。')
    elif age >= 20:
        print('可看輔級電影和投票。')
    elif age >= 40:
        print('可看輔級電影、投票和選總統。')
    elif age >= 65:   # 最specific(範圍最窄)
        print('可看輔級電影、投票、選總統和退休。')
    else:
        print('年齡太小，長大再說。')

10歲：年齡太小，長大再說。
12歲：可看輔級電影。
20歲：可看輔級電影。
39歲：可看輔級電影。
40歲：可看輔級電影。
67歲：可看輔級電影。


In [ ]:
# 從最general到最specific的寫法錯誤。
ages = (10, 12, 20, 39, 40, 67)
for age in ages:
    print(f'{age}歲：', end='')
    if age >= 12:   # 最general(範圍最寬)
        print('可看輔級電影。')
    elif age >= 20:
        print('可看輔級電影和投票。')
    elif age >= 40:
        print('可看輔級電影、投票和選總統。')
    elif age >= 65:   # 最specific(範圍最窄)
        print('可看輔級電影、投票、選總統和退休。')
    else:
        print('年齡太小，長大再說。')

10歲：年齡太小，長大再說。
12歲：可看輔級電影。
20歲：可看輔級電影。
39歲：可看輔級電影。
40歲：可看輔級電影。
67歲：可看輔級電影。


In [94]:
# 這樣還是錯。
ages = (10, 12, 20, 39, 40, 67)
for age in ages:
    print(f'{age}歲：', end='')
    if age < 999:   # 最general(範圍最寬)
        print('可看輔級電影、投票、選總統和退休。')
    elif age < 65:   
        print('可看輔級電影、投票和選總統。')
    elif age < 40:
        print('可看輔級電影和投票。')
    elif age < 20:
        print('可看輔級電影。')
    else:   # 最specific(範圍最窄)
        print('年齡太小，長大再說。')

10歲：可看輔級電影、投票、選總統和退休。
12歲：可看輔級電影、投票、選總統和退休。
20歲：可看輔級電影、投票、選總統和退休。
39歲：可看輔級電影、投票、選總統和退休。
40歲：可看輔級電影、投票、選總統和退休。
67歲：可看輔級電影、投票、選總統和退休。


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

### <font color='DarkSalmon'><b>修訂為「先特殊後普通」</b></font>

* 以下兩種寫法都是正確的，可擇一而用。

In [ ]:
# 從最specific到最general的寫法才正確。
ages = (10, 12, 20, 39, 40, 67)
for age in ages:
    print(f'{age}歲：', end='')
    if age >= 65:   # 最specific(範圍最窄)
        print('可看輔級電影、投票、選總統和退休。')
    elif age >= 40:
        print('可看輔級電影、投票和選總統。')
    elif age >= 20:
        print('可看輔級電影和投票。')
    elif age >= 12:   # 最general(範圍最寬)
        print('可看輔級電影。')
    else:
        print('年齡太小，長大再說。')

In [ ]:
# 從最specific到最general的另一種寫法。
ages = (10, 12, 20, 39, 40, 67)
for age in ages:
    print(f'{age}歲：', end='')
    if age < 12:  # 最specific  
        print('年齡太小，長大再說。')
    elif age < 20:
        print('可看輔級電影。')
    elif age < 40:
        print('可看輔級電影和投票。')
    elif age < 65:  # 最general
        print('可看輔級電影、投票和選總統。')
    else:
        print('可看輔級電影、投票、選總統和退休。')
        

<div style="text-align:center"><img src="https://i.imgur.com/wh9tSJ6.jpg" width="650"/></div>

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

### <font color='DarkSalmon'><b>Example-2</b></font>

* 某會所根據會員上年度消費金額，畫分為不同等級：

    <div style="font-size: 80%;">

    |消費金額(元)|會員等級|
    |--|--|
    | 0-10000        |  奇石會員|
    | 10001-100000   |  翡翠會員|
    | 100001-500000  |  珍珠會員|
    | 500001-1000000 |  黃金會員|
    | 1000001-5000000|  白金會員|
    | 5000001以上   |鑽石會員|

In [ ]:
# 錯誤寫法：
def get_member_level_gen2spec(amount: int) -> str:
    if amount >= 0:    # most general(範圍最寬)
        member_level = '奇石會員'
    elif amount >= 10001:
        member_level = '翡翠會員'
    elif amount >= 100001:
        member_level = '珍珠會員'
    elif amount >= 500001:
        member_level = '黃金會員'
    elif amount >= 1000001:
        member_level = '白金會員'
    else:              # most specific(範圍最窄)
        member_level = '鑽石會員'
    return member_level        

# 正確寫法-1
def get_member_level_spec2gen_1(amount: int) -> str:
    if amount <= 10000:   # most specific(範圍最窄)
        member_level = '奇石會員'
    elif amount <= 100000:
        member_level = '翡翠會員'
    elif amount <= 500000:
        member_level = '珍珠會員'
    elif amount <= 1000000:
        member_level = '黃金會員'
    elif amount <= 5000000:
        member_level = '白金會員'
    else:                 # most general(範圍最寬)
        member_level = '鑽石會員'
    return member_level   

# 正確寫法-2        
def get_member_level_spec2gen_2(amount: int) -> str:
    if amount >= 5000001:    # most specific(範圍最窄)
        member_level = '鑽石會員'
    elif amount >= 1000001:
        member_level = '白金會員'
    elif amount >= 500001:
        member_level = '黃金會員'
    elif amount >= 100001:
        member_level = '珍珠會員'
    elif amount >= 10001:
        member_level = '翡翠會員'
    else:                    # most general(範圍最寬)
        member_level = '奇石會員'
    return member_level                
        
amounts = (10_000, 90_000, 350_000, 500_001, 4_999_999, 30_000_000)
for amount in amounts:
    print(f'general to specific  : {amount:,} {get_member_level_gen2spec(amount)}')
    print(f'specific to general-1: {amount:,} {get_member_level_spec2gen_1(amount)}')
    print(f'specific to general-2: {amount:,} {get_member_level_spec2gen_2(amount)}\n')

<div style="text-align:center"><img src="https://i.imgur.com/KqDhjxW.jpg" width="650"/></div>

<div style="text-align:center"><img src="" width="650"/></div>